In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
##
## Row representa una fila en un RDD
#
from pyspark.sql import Row
##
## Crea un DataFrame a partir del archivo con
## formato CSV
##
nombre_archivo="icfes_taller"
df = spark.read.load("D:\\CLASES\\ELECTIVA 3 BigData\\Taller_final\\"+nombre_archivo+".csv",
                     format="csv",
                     sep="|",
                     inferSchema= True,
                     encoding='UTF-8',
                     decimal=".",
                     header="true")
df.printSchema()

root
 |-- estu_edad: double (nullable = true)
 |-- estu_genero: string (nullable = true)
 |-- estu_nacimiento_dia: double (nullable = true)
 |-- estu_nacimiento_mes: double (nullable = true)
 |-- estu_nacimiento_anno: double (nullable = true)
 |-- estu_zona_reside: double (nullable = true)
 |-- estu_area_reside: double (nullable = true)
 |-- cole_valor_pension: double (nullable = true)
 |-- estu_trabaja: double (nullable = true)
 |-- fami_estrato_vivienda: double (nullable = true)
 |-- estu_veces_estado: double (nullable = true)
 |-- fami_educa_padre: double (nullable = true)
 |-- fami_educa_madre: double (nullable = true)
 |-- fami_ocup_padre: double (nullable = true)
 |-- fami_ocup_madre: double (nullable = true)
 |-- fami_nivel_sisben: double (nullable = true)
 |-- fami_pisos_hogar: double (nullable = true)
 |-- fami_personas_hogar: double (nullable = true)
 |-- fami_telefono_fijo: double (nullable = true)
 |-- fami_celular: double (nullable = true)
 |-- fami_internet: double (nulla

In [6]:
df.select("estu_edad","prom_puntaje","cole_cod_icfes",'cole_nombre_sede').where("cole_cod_icfes=39255").show(1000)

+---------+------------------+--------------+--------------------+
|estu_edad|      prom_puntaje|cole_cod_icfes|    cole_nombre_sede|
+---------+------------------+--------------+--------------------+
|     16.0|          50.42875|         39255|INSTITUCION EDUCA...|
|     17.0|43.397499999999994|         39255|INSTITUCION EDUCA...|
|     16.0|58.356249999999996|         39255|INSTITUCION EDUCA...|
|     16.0|           52.1875|         39255|INSTITUCION EDUCA...|
|     17.0|59.401250000000005|         39255|INSTITUCION EDUCA...|
|     17.0|56.459999999999994|         39255|INSTITUCION EDUCA...|
|     19.0|          46.66625|         39255|INSTITUCION EDUCA...|
|     16.0|45.443749999999994|         39255|INSTITUCION EDUCA...|
|     18.0|             47.77|         39255|INSTITUCION EDUCA...|
|     16.0|              49.2|         39255|INSTITUCION EDUCA...|
|     19.0| 38.64999999999999|         39255|INSTITUCION EDUCA...|
|     18.0|50.411249999999995|         39255|INSTITUCION EDUCA

In [12]:
df.createOrReplaceTempView('mi_tabla')
sql=("select estu_zona_reside,min(fami_estrato_vivienda),max(fami_estrato_vivienda),avg(fami_estrato_vivienda) prom from"
     " mi_tabla group by estu_zona_reside" )
spark.sql(sql).show()

+----------------+--------------------------+--------------------------+--------------------+
|estu_zona_reside|min(fami_estrato_vivienda)|max(fami_estrato_vivienda)|                prom|
+----------------+--------------------------+--------------------------+--------------------+
|             8.0|                       1.0|                       3.0|                 2.5|
|             0.0|                       0.0|                       1.0|0.014925373134328358|
|             7.0|                       1.0|                       5.0|  1.5263157894736843|
|             1.0|                       0.0|                       3.0|  1.6666666666666667|
|             4.0|                       0.0|                       5.0|  1.7777777777777777|
|             3.0|                       2.0|                       3.0|                 2.4|
|             2.0|                       1.0|                       3.0|  1.6666666666666667|
|            10.0|                       0.0|               

In [19]:
sql1=("select estu_zona_reside,min(fami_estrato_vivienda),max(fami_estrato_vivienda),avg(fami_estrato_vivienda) from"
    + " mi_tabla group by estu_zona_reside" 
    +" having avg(fami_estrato_vivienda)=(select max(t1.prom)"
    +" from (select estu_zona_reside,avg(fami_estrato_vivienda) prom from"
     +" mi_tabla group by estu_zona_reside)t1)" )

spark.sql(sql1).show()

+----------------+--------------------------+--------------------------+--------------------------+
|estu_zona_reside|min(fami_estrato_vivienda)|max(fami_estrato_vivienda)|avg(fami_estrato_vivienda)|
+----------------+--------------------------+--------------------------+--------------------------+
|             8.0|                       1.0|                       3.0|                       2.5|
+----------------+--------------------------+--------------------------+--------------------------+



In [15]:
sql1

'select estu_zona_reside,min(fami_estrato_vivienda),max(fami_estrato_vivienda),avg(fami_estrato_vivienda) from mi_tabla group by estu_zona_reside having avg(fami_estrato_vivienda)=(  select max(t1.prom) from (select estu_zona_reside,min(fami_estrato_vivienda),max(fami_estrato_vivienda),avg(fami_estrato_vivienda) from mi_tabla group by estu_zona_reside))t1'

In [8]:
df.createOrReplaceTempView('icfesview')
spark .sql("select cole_cod_icfes,cole_nombre_sede,prom_puntaje from icfesview where prom_puntaje = (select max(prom_puntaje) from icfesview)").show()


+--------------+--------------------+-----------------+
|cole_cod_icfes|    cole_nombre_sede|     prom_puntaje|
+--------------+--------------------+-----------------+
|         11882|COLEGIO NACIONAL ...|86.18500000000002|
+--------------+--------------------+-----------------+

